In [25]:
import os

In [26]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain

In [ ]:
TEMPLATE = """
Text:{text}
You are an expert mcq maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your responses like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs
###RESPONSE_JSON
{response_json}
"""

In [ ]:
RESPONSE_JSON = {
    "1":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"
        },
        "correct":"correct answer",
    },
    "2":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"
        },
        "correct":"correct answer",
    },
    "3":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"
        },
        "correct":"correct answer",
    },
    "4":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"
        },
        "correct":"correct answer",
    },
    "5":{
        "mcq":"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here"
        },
        "correct":"correct answer",
    },

}

In [27]:
prompt = PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
)

In [28]:
t5llm = HuggingFaceHub(repo_id='facebook/mbart-large-50', model_kwargs={'temperature':1.5})

In [29]:
chain = LLMChain(llm=t5llm,prompt = prompt)

In [30]:
chain.run("colorful clothes")

'What is a good name for a company that makes colorful clothes'